In [ ]:
import sys # for automation and parallelization
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])
if manual:
    %matplotlib inline

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from quetzal.model import stepmodel
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
# import helper functions
# imports all helper functions stored in class "helpers"
%run -i helperFunctions.py

# Create zones and add basic information
## Zones Berliner Ortsteile, number of inhabitants
Uses Berliner Ortsteile (n=98), source: https://fbinter.stadt-berlin.de/fb/wfs/data/senstadt/s_wfs_alkis_ortsteile

Population data on bulding block level 2020, source: https://fbinter.stadt-berlin.de/fb/wfs/data/senstadt/s06_06ewdichte2020

In [ ]:
input_path = '../input/zones/'
output_path = '../output/'
model_path = '../model/'

In [ ]:
input_path = '../input/zones/'
output_path = '../output/'
model_path = '../model/'


In [ ]:
# Initialising StepModel...
sm = stepmodel.StepModel(debug=False,
                         walk_on_road=False,
                         epsg=4326,
                         coordinates_unit=None)

### read in Berliner Ortsteile shapefile

In [ ]:
# Adding zones...
zones = gpd.read_file(input_path + "Berliner_Ortsteile.shp")
# remove duplicate zone (Buch)
zones = zones[list(map(lambda x: x not in zones[zones["nam"]=="Buch"][1:].index, zones.index))]
zones = zones.reset_index()
zones["zoneIndex"] = zones.index
sm.zones = zones
zones.plot()

### read car free zone border

In [ ]:
# adding border
car_border = gpd.read_file(input_path + "Umweltzone.shp")

#plot
fig, ax = plt.subplots(figsize = (16,16))
zones.plot(ax=ax, color = "#ffffff33", edgecolor="#999999")
car_border.plot(ax=ax,color='#ffffff33',  edgecolor = "red")

In [ ]:
zones_union = zones.unary_union
#zones_union.plot()

middle = gpd.GeoSeries(zones.geometry.intersection(car_border.unary_union))
outter = gpd.GeoSeries(zones.geometry.difference(car_border.unary_union))
whole_with_car_border = middle.geometry.append(outter.geometry)

fig, ax = plt.subplots(figsize = (16,16))
outter.plot(ax=ax, color = '#dc8f8f')
middle.plot(ax=ax, color='#3DC76D')

whole_with_car_border.plot( ax=ax, color = "#ffffff33", edgecolor="#999999")

### Open Questions:
- Join smaller bits into bigger zones?
- Use only for car-free or also for 'before' (Anja: for both)